In [1]:
import os, sys

# 1. Compute the src/ directory (one level up from notebooks/)
notebooks_dir = os.getcwd()                            # e.g. …/graphworld/src/notebooks
src_dir       = os.path.abspath(os.path.join(notebooks_dir, os.pardir))
# 2. Prepend it to Python’s module search path
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

# 3. Now you can import as expected
from graph_world.models.basic_gnn import HGCN
print("✅ Imported HGCN from", HGCN)

/Users/daniel/miniconda3/envs/graphworld/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Imported HGCN from <class 'graph_world.models.basic_gnn.HGCN'>


In [2]:
import time
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_undirected

from graph_world.models.basic_gnn import GCN, HGCN

# 1) Load Cora
dataset = Planetoid(root='.', name='Cora')
data = dataset[0]
data.edge_index = to_undirected(data.edge_index)
device = torch.device("cpu") 
data = data.to(device)

def train_model(model, data, epochs=200, lr=0.01, weight_decay=5e-4):
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    model.train()
    for _ in range(epochs):
        opt.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        opt.step()
    return model

def test_model(model, data):
    model.eval()
    logits = model(data.x, data.edge_index)
    pred = logits.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        accs.append((pred[mask] == data.y[mask]).float().mean().item())
    return accs  # train, val, test

def param_count(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# 2) Instantiate & run GCN
gcn = GCN(in_channels=dataset.num_node_features,
          hidden_channels=16,
          num_layers=2,
          out_channels=dataset.num_classes,
          dropout=0.5).to(device)

t0 = time.time()
gcn = train_model(gcn, data)
gcn_time = time.time() - t0
gcn_accs = test_model(gcn, data)

print(f"GCN params: {param_count(gcn):,}  train/val/test = {gcn_accs}  time = {gcn_time:.2f}s")

# 3) Instantiate & run HGCN
hgcn = HGCN(in_channels=dataset.num_node_features,
            hidden_channels=16,
            num_layers=2,
            out_channels=dataset.num_classes,
            c=None,            # learnable curvatures
            manifold='Hyperboloid',
            dropout=0.5,
            bias=True,
            act_name='relu').to(device)

t0 = time.time()
hgcn = train_model(hgcn, data)
hgcn_time = time.time() - t0
hgcn_accs = test_model(hgcn, data)

print(f"HGCN hyperboloid params: {param_count(hgcn):,}  train/val/test = {hgcn_accs}  time = {hgcn_time:.2f}s")


GCN(1433, 7, num_layers=2)
GCN params: 23,335  train/val/test = [1.0, 0.7200000286102295, 0.7450000047683716]  time = 0.91s
HGCN(in=1433, hid=16, layers=2, out=7, manifold=Hyperboloid, c = None)
HGCN hyperboloid params: 23,354  train/val/test = [1.0, 0.7360000014305115, 0.75]  time = 12.20s
